In [ ]:
%pip install -r ../requirements.txt

In [1]:
import sys
from pathlib import Path
import pandas as pd

# Add ML/ to Python path so "import src...." works
ML_ROOT = Path.cwd().parent  # notebooks/ -> ML/
if str(ML_ROOT) not in sys.path:
    sys.path.insert(0, str(ML_ROOT))

from src.config import MODELS_DIR, GOLD_DIR, MLFLOW_EXPERIMENT_NAME, MLFLOW_TRACKING_URI

from src.uc1_late_risk.build_scoring_set import build_scoring_set

from src.uc1_late_risk.inference import (
    load_bundle_by_model_id,
    load_bundle_from_registry,
    score_csv,     
    write_explanations_jsonl
)


In [2]:
scoring_csv_path = build_scoring_set(
    scoring_date=None,            # today
    output_csv=GOLD_DIR / "uc1_scoring_set.csv",
    include_ids=True
)

scoring_csv_path

anchor_date: 2026-01-17
scoring_set rows: 299
saved to: C:\Users\PC\Documents\DXC Hackathon\bnpl-intelligent-analytics\data\gold\uc1_scoring_set.csv


WindowsPath('C:/Users/PC/Documents/DXC Hackathon/bnpl-intelligent-analytics/data/gold/uc1_scoring_set.csv')

In [3]:
REGISTERED_MODEL_NAME = "uc1_late_payment_risk"
bundle, model_uri = load_bundle_from_registry(
    registered_model_name=REGISTERED_MODEL_NAME,
    alias="champion",
    tracking_uri=MLFLOW_TRACKING_URI
)

print("Loaded from Registry:", model_uri)
print("bundle.model_id:", bundle.model_id)
print("bundle.model_name:", bundle.model_name)
print("threshold:", bundle.risk_threshold)

c:\Users\PC\AppData\Local\Programs\Python\Python314\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
c:\Users\PC\AppData\Local\Programs\Python\Python314\Lib\site-packages\mlflow\tracking\_model_registry\utils.py:216: FutureWarning: The filesystem model registry backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guid

Loaded from Registry: models:/uc1_late_payment_risk@champion
bundle.model_id: uc1_late_payment_risk
bundle.model_name: LogReg
threshold: 0.48999999999999994


In [4]:
out_csv = GOLD_DIR / "uc1_scored_today.csv"

score_csv(
    bundle=bundle,
    input_csv=scoring_csv_path,
    output_csv=out_csv
)

pd.read_csv(out_csv).head()


,installment_id,order_id,user_id,merchant_id,installment_number,anchor_date,due_date,status,account_age_days,kyc_level_num,...,merchant_refund_rate_90d,merchant_risk_score,category,city_merchant,model_id,model_name,proba_late_30d,risk_threshold,is_risky_late,margin_vs_threshold
0,inst_0000002,order_000006,user_00002,merchant_0109,2,2026-01-17,2026-03-02,unpaid,71.0,2,...,0.0,0.000000,electronics,Casablanca,uc1_late_payment_risk,LogReg,0.305089,0.49,0,-0.184911
1,inst_0000018,order_000014,user_00004,merchant_0061,3,2026-01-17,2026-03-05,unpaid,120.0,2,...,0.0,0.000000,fashion,Casablanca,uc1_late_payment_risk,LogReg,0.387056,0.49,0,-0.102944
2,inst_0000032,order_000020,user_00006,merchant_0096,2,2026-01-17,2026-02-02,unpaid,84.0,2,...,0.0,0.086957,travel,Rabat,uc1_late_payment_risk,LogReg,0.250298,0.49,0,-0.239702
3,inst_0000039,order_000022,user_00007,merchant_0095,3,2026-01-17,2026-05-23,unpaid,44.0,2,...,0.0,0.000000,electronics,Rabat,uc1_late_payment_risk,LogReg,0.198786,0.49,0,-0.291214
4,inst_0000041,order_000023,user_00007,merchant_0041,2,2026-01-17,2026-04-05,unpaid,44.0,2,...,0.0,0.000000,electronics,Casablanca,uc1_late_payment_risk,LogReg,0.183719,0.49,0,-0.306281


In [5]:
df_scoring = pd.read_csv(scoring_csv_path)

write_explanations_jsonl(
    bundle=bundle,
    df_scoring=df_scoring,
    out_jsonl="../../data/scoring/uc1_explanations_today.jsonl",
    only_risky=True,
    id_cols=["installment_id","user_id","order_id","merchant_id"],
    base_limit=4000.0,
    top_k=8,
    request_id_prefix="today"
)


[write_explanations_jsonl] wrote 38 payloads to ..\..\data\scoring\uc1_explanations_today.jsonl


WindowsPath('../../data/scoring/uc1_explanations_today.jsonl')